In [1]:
import h5py
import keras
import numpy as np
import json
import os
import uuid
import yaml

from attlayer import AttentionWeightedAverage
#from avglayer import MaskAverage
from copy import deepcopy
#from finetuning import (sampling_generator, finetuning_callbacks)
from operator import itemgetter
#from global_variables import NB_TOKENS, NB_EMOJI_CLASSES
from keras.layers import *
from keras.layers.merge import concatenate
from keras.layers import Input, Bidirectional, Embedding, Dense, Dropout, SpatialDropout1D, LSTM, Activation
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.regularizers import L1L2 
from pathlib import Path
from sklearn.metrics import classification_report, recall_score, precision_score, f1_score
from os.path import exists

Using TensorFlow backend.


In [2]:
def elsa_architecture(nb_classes, nb_tokens, maxlen, feature_output=False, embed_dropout_rate=0, final_dropout_rate=0, embed_dim=300,
                    embed_l2=1E-6, return_attention=False, load_embedding=False, pre_embedding=None, high=False, LSTM_hidden=512, LSTM_drop=0.5):
    """
    Returns the DeepMoji architecture uninitialized and
    without using the pretrained model weights.
    # Arguments:
        nb_classes: Number of classes in the dataset.
        nb_tokens: Number of tokens in the dataset (i.e. vocabulary size).
        maxlen: Maximum length of a token.
        feature_output: If True the model returns the penultimate
                        feature vector rather than Softmax probabilities
                        (defaults to False).
        embed_dropout_rate: Dropout rate for the embedding layer.
        final_dropout_rate: Dropout rate for the final Softmax layer.
        embed_l2: L2 regularization for the embedding layerl.
        high: use or not the highway network
    # Returns:
        Model with the given parameters.
    """
    class NonMasking(Layer):   
        def __init__(self, **kwargs):   
            self.supports_masking = True  
            super(NonMasking, self).__init__(**kwargs)   

        def build(self, input_shape):   
            input_shape = input_shape   

        def compute_mask(self, input, input_mask=None):   
            # do not pass the mask to the next layers   
            return None   

        def call(self, x, mask=None):   
            return x   

        def get_output_shape_for(self, input_shape):   
            return input_shape 
    # define embedding layer that turns word tokens into vectors
    # an activation function is used to bound the values of the embedding
    model_input = Input(shape=(maxlen,), dtype='int32')
    embed_reg = L1L2(l2=embed_l2) if embed_l2 != 0 else None

    if not load_embedding and pre_embedding is None:
        embed = Embedding(input_dim=nb_tokens, output_dim=embed_dim, mask_zero=True,input_length=maxlen,embeddings_regularizer=embed_reg,
                          name='embedding')
    else:
        embed = Embedding(input_dim=nb_tokens, output_dim=embed_dim, mask_zero=True,input_length=maxlen, weights=[pre_embedding],
                          embeddings_regularizer=embed_reg,trainable=True, name='embedding')
    if high:
        x = NonMasking()(embed(model_input))
    else:
        x = embed(model_input)
    x = Activation('tanh')(x)

    # entire embedding channels are dropped out instead of the
    # normal Keras embedding dropout, which drops all channels for entire words
    # many of the datasets contain so few words that losing one or more words can alter the emotions completely
    if embed_dropout_rate != 0:
        embed_drop = SpatialDropout1D(embed_dropout_rate, name='embed_drop')
        x = embed_drop(x)

    # skip-connection from embedding to output eases gradient-flow and allows access to lower-level features
    # ordering of the way the merge is done is important for consistency with the pretrained model
    lstm_0_output = Bidirectional(LSTM(LSTM_hidden, return_sequences=True, dropout=LSTM_drop), name="bi_lstm_0" )(x)
    lstm_1_output = Bidirectional(LSTM(LSTM_hidden, return_sequences=True, dropout=LSTM_drop), name="bi_lstm_1" )(lstm_0_output)
    x = concatenate([lstm_1_output, lstm_0_output, x])
    if high:
        x = TimeDistributed(Highway(activation='tanh', name="high"))(x)
    # if return_attention is True in AttentionWeightedAverage, an additional tensor
    # representing the weight at each timestep is returned
    weights = None
    x = AttentionWeightedAverage(name='attlayer', return_attention=return_attention)(x)
    #x = MaskAverage(name='attlayer', return_attention=return_attention)(x)
    if return_attention:
        x, weights = x

    if not feature_output:
        # output class probabilities
        if final_dropout_rate != 0:
            x = Dropout(final_dropout_rate)(x)

        if nb_classes > 2:
            outputs = [Dense(nb_classes, activation='softmax', name='softmax')(x)]
        else:
            outputs = [Dense(1, activation='sigmoid', name='softmax')(x)]
    else:
        # output penultimate feature vector
        outputs = [x]

    if return_attention:
        # add the attention weights to the outputs if required
        outputs.append(weights)

    return Model(inputs=[model_input], outputs=outputs)

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
cur_lan = "elsa_de"
maxlen = 20
batch_size = 250
lr = 3e-4
epoch_size = 25000
nb_epochs = 1000
patience = 5
checkpoint_weight_path = "./ckpt/weights.{epoch:02d}-{val_loss:.3f}_de.hdf5"
loss = "categorical_crossentropy"
optim = "adam"
vocab_path = "/data/elsa"
nb_classes=64

In [4]:
LSTM_hidden = 512
LSTM_drop = 0.5
final_dropout_rate = 0.5
embed_dropout_rate = 0.0
high = False
load_embedding = True
embed_dim = 200

In [5]:
steps = int(epoch_size/batch_size)

wv_path = Path(vocab_path).joinpath("{:s}_wv.npy".format(cur_lan)).as_posix()
X_path = Path(vocab_path).joinpath("{:s}_X.npy".format(cur_lan)).as_posix()
y_path = Path(vocab_path).joinpath("{:s}_y.npy".format(cur_lan)).as_posix()

word_vec = np.load(wv_path, allow_pickle=True)
input_vec, input_label = np.load(X_path, allow_pickle=True), np.load(y_path, allow_pickle=True)
nb_tokens, input_len = len(word_vec), len(input_label)

train_end = int(input_len*0.7)
val_end = int(input_len*0.9)

(X_train, y_train) = (input_vec[:train_end], input_label[:train_end])
(X_val, y_val) = (input_vec[train_end:val_end], input_label[train_end:val_end])
(X_test, y_test) = (input_vec[val_end:], input_label[val_end:])

In [6]:
model = elsa_architecture(nb_classes=nb_classes, nb_tokens=nb_tokens, maxlen=maxlen, final_dropout_rate=final_dropout_rate, embed_dropout_rate=embed_dropout_rate, 
                          load_embedding=True, pre_embedding=word_vec, high=high, embed_dim=embed_dim)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 20, 200)      7720600     input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 20, 200)      0           embedding[0][0]                  
__________________________________________________________________________________________________
bi_lstm_0 (Bidirectio

In [7]:
if optim == 'adam':
    adam = Adam(clipnorm=1, lr=lr)
    model.compile(loss=loss, optimizer=adam, metrics=['accuracy'])
elif optim == 'rmsprop':
    model.compile(loss=loss, optimizer='rmsprop', metrics=['accuracy'])

model.fit(X_train,
          y_train,
          batch_size=batch_size,
          epochs=nb_epochs,
          validation_data=(X_val, y_val),
          callbacks=[
              keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=patience, verbose=0, mode='auto'),
              keras.callbacks.ModelCheckpoint(checkpoint_weight_path, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
          ],
          verbose=True)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 32351 samples, validate on 9243 samples
Epoch 1/1000
32351/32351 [==============================] - 29s 897us/step - loss: 3.7964 - acc: 0.1293 - val_loss: 3.6030 - val_acc: 0.1438
Epoch 2/1000
32351/32351 [==============================] - 23s 721us/step - loss: 3.4925 - acc: 0.1661 - val_loss: 3.3760 - val_acc: 0.1823
Epoch 3/1000
32351/32351 [==============================] - 23s 698us/step - loss: 3.3140 - acc: 0.1903 - val_loss: 3.2960 - val_acc: 0.1944
Epoch 4/1000
32351/32351 [==============================] - 22s 676us/step - loss: 3.2069 - acc: 0.2057 - val_loss: 3.2411 - val_acc: 0.2012
Epoch 5/1000
32351/32351 [==============================] - 22s 671us/step - loss: 3.1202 - acc: 0.2213 - val_loss: 3.2277 - val_acc: 0.2083
Epoch 6/1000
32351/32351 [==============================] - 22s 682us/step - loss: 3.0367 - acc: 0.2362 - val_loss: 3.21

In [8]:
_, acc = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)

print(acc)

0.1990480327092946


In [9]:
token2index = json.loads(open("/data/elsa/elsa_de_vocab.txt", "r").read())

freq = {line.split()[0]: int(line.split()[1]) for line in open("/data/elsa/elsa_de_emoji.txt").readlines()}
freq_topn = sorted(freq.items(), key=itemgetter(1), reverse=True)[:nb_classes]

In [10]:
y_pred = model.predict(X_test)
print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1), target_names=[e[0] for e in freq_topn]))

              precision    recall  f1-score   support

           😂       0.22      0.57      0.32       599
           😍       0.22      0.27      0.24       233
           ❤       0.12      0.17      0.14       183
           👍       0.17      0.21      0.19       166
           😊       0.11      0.23      0.15       207
           🤣       0.17      0.01      0.01       136
           😉       0.11      0.21      0.14       210
           😘       0.19      0.21      0.20       123
           🤔       0.22      0.28      0.25       192
           😭       0.30      0.18      0.22       114
           😁       0.06      0.01      0.01       149
           ☕       0.26      0.55      0.35        65
           😅       0.07      0.03      0.04       134
           😎       0.04      0.02      0.03        98
           🙄       0.07      0.05      0.06       115
           ⭐       0.64      0.67      0.66        67
           🙈       0.02      0.01      0.01        87
           ♂       0.20    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
input_vec.shape

(46216, 20)